In [605]:
from src.data_processing import data_processing, data_dropping
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, Lasso, LinearRegression
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, make_scorer, accuracy_score, mean_squared_error
from nltk.stem import WordNetLemmatizer as Lemmatizer
from collections import Counter
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
import re

In [494]:
with open('data/predictor.p', 'rb') as file:
    predictor = pickle.load(file)
with open('data/target.p', 'rb') as file:
    target = pickle.load(file)

In [602]:
def hasnumber(s):
    return bool(re.search(r'\d', s))

In [495]:
pred = data_processing(data_dropping(predictor))

In [496]:
pred = pred[['existing_stories', 'existing_use', 'proposed_use', 'description']]

In [14]:
pred['has_existing_stories'] = (pred['existing_stories'].fillna(0) > 0).astype(int)

C:\Users\pengj\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [497]:
pred[['existing_use', 'proposed_use', 'description']] = pred[['existing_use', 'proposed_use', 'description']].fillna('-').astype(str)

In [498]:
pred['text'] = pred[['existing_use', 'proposed_use', 'description']].apply(lambda x: ' '.join(x), axis = 1)

In [581]:
float('29')

29.0

In [474]:
with open('data/pred_trimmed.p', 'wb') as file:
    pickle.dump(pred, file)

In [603]:
text = pred['text'].values

In [623]:
y_full = (target['fire_rate_after'] > target['fire_rate_before']).astype(int).values

In [582]:
def int_able(str):
    try:
        float(str)
    except:
        return False
    return True

In [606]:
text = np.array([' '.join([Lemmatizer.lemmatize('n', word.lower()) for word in doc.split() if not hasnumber(word)]) for doc in text])

In [607]:
text_lemmatized = text

NameError: name 'isnum' is not defined

In [643]:
n = 50000
sample_idx = np.random.choice(len(pred), n)

In [644]:
X = text[sample_idx]
y = y_full[sample_idx]

In [439]:
train = np.full(n, False)
train_idx = np.random.choice(n, int(.75*n), replace = False)
for idx in train_idx:
    train[idx] = True
test = ~train

In [563]:
vec = TfidfVectorizer(stop_words = 'english')

In [645]:
X = vec.fit_transform(X)

In [625]:
model = LogisticRegression()

In [646]:
model.fit(X[train],y[train])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [647]:
model.score(X[test], y[test])

0.8596

In [648]:
roc_auc_score(y[test], model.predict_proba(X[test])[:,1])

0.8290845748248943

In [619]:
vocab_idx[13029]

'spinkler'

In [615]:
model.feature_importances_.argsort()

array([    0, 10394, 10395, ...,  6553, 13029, 12942], dtype=int64)

In [616]:
roc_auc_score(y[test], model.predict_proba(X[test])[:,1])

AttributeError: 'GradientBoostingRegressor' object has no attribute 'predict_proba'

In [506]:
def f1(y_test, y_pred_prob, threshold):
    y_pred = (y_pred_prob > threshold).astype(int)
    return f1_score(y_test, y_pred)

In [507]:
current_best = 0
best_i = 0
for i in range(1000):
    f = f1(y[test], model.predict_proba(X[test])[:,1], i/1000)
    if f > current_best:
        current_best = f
        best_i = i
    

C:\Users\pengj\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [508]:
f1(y[test], model.predict_proba(X[test])[:,1], best_i/1000)

0.43174820390010266

In [509]:
vec.vocabulary_;

In [649]:
vocab_idx = {v: k for k, v in vec.vocabulary_.items()}

In [650]:
important_words = {}

In [651]:
for i, coef in enumerate(model.coef_[0]):
    if coef > 0:
        important_words[vocab_idx[i]] = coef

In [652]:
c = Counter(important_words)

In [658]:
model.intercept_

array([-1.62033885])

In [654]:
top_increase_2

[('maher', 4.135960726805149),
 ('apt', 3.2571120896300103),
 ('traps', 2.529927485747275),
 ('ordinance', 2.5289745837396778),
 ('speaker', 2.271684087536629),
 ('egress', 2.095901081992529),
 ('ref', 2.062200879369185),
 ('mep', 2.044750346887388),
 ('exh', 2.032254322337327),
 ('damage', 2.0148565235091977),
 ('modification', 2.0038582407406706),
 ('deferred', 2.001935973309281),
 ('evacuation', 1.9743179655078928),
 ('reroof', 1.9415711275179024),
 ('fiberglass', 1.813847341062362),
 ('occupied', 1.7999530375657145),
 ('community', 1.7843406873482186),
 ('maintain', 1.7662882423747037),
 ('sleeping', 1.7617268526760557),
 ('priority', 1.7045146788140375)]

In [657]:
top_increase_1

[('maher', 4.318628315280367),
 ('traps', 3.0604753135583738),
 ('apt', 3.058518434175814),
 ('lobby', 2.4061483916803845),
 ('mep', 2.38283168170562),
 ('obtain', 2.229604895446058),
 ('evacuation', 2.2222105640105316),
 ('ti', 2.197495985636132),
 ('hotel', 2.0324178132562194),
 ('hospital', 2.0289820845986837),
 ('altered', 1.9812141589177144),
 ('renovation', 1.959745311045452),
 ('pendent', 1.9546521857902992),
 ('occupied', 1.9066297627457913),
 ('enclosure', 1.9050113978818335),
 ('common', 1.8730919602814982),
 ('ordinance', 1.868788989507143),
 ('assembly', 1.799356956704036),
 ('fan', 1.7938171125364328),
 ('efficiency', 1.7586792977760355)]

In [515]:
top10_decrease_2

[('cti', 8.568333416450177),
 ('105', 4.519257013734376),
 ('coutnertop', 4.345246902269198),
 ('oti', 3.3374166992264547),
 ('drinking', 3.262354378056696),
 ('screens', 3.087552444844313),
 ('density', 2.990672860273146),
 ('addressable', 2.9302821775207684),
 ('winder', 2.9110521360872648),
 ('fluorescent', 2.8903632859330872)]

In [381]:
important_words_4 = important_words

In [392]:
important_words;

In [415]:
top10_2 = c.most_common(10)

In [401]:
top10_1 = c.most_common(10)

In [433]:
top10_3

[('traps', 9.066135535601088),
 ('exh', 5.236541100818746),
 ('265', 4.812601390528742),
 ('maher', 4.730539830504769),
 ('tags', 4.297665411723898),
 ('geotech', 4.20148706744233),
 ('901', 4.104538395668117),
 ('9884', 4.058352578295828),
 ('notifier', 3.8099096402798533),
 ('144', 3.680255661760993)]

In [416]:
top10_2

[('traps', 7.005050918380217),
 ('maher', 4.692009685187687),
 ('forms', 3.636798151040242),
 ('ref', 3.2956199551363703),
 ('evacuation', 2.8559641655760863),
 ('apt', 2.763896991448432),
 ('assmbly', 2.759404362814315),
 ('3884sf', 2.631019912856813),
 ('suite', 2.5307707835609),
 ('unit', 2.5235550187500255)]

In [417]:
top10_1

[('traps', 6.190488019758687),
 ('maher', 4.546629916650752),
 ('uprights', 3.1050460402905613),
 ('daycare', 2.8857078892696255),
 ('sounders', 2.7342078862780332),
 ('altered', 2.622869978271953),
 ('evacuation', 2.6174622302659016),
 ('washer', 2.552283019923069),
 ('arch', 2.4326521113122124),
 ('chap', 2.366688547050922)]

In [641]:

for i, row in pred[pred['description'].apply(lambda x: ' MAHER ' in x)].reset_index().iterrows():
    print(row['description'])
    if i > 10:
        break

NEW EXTERIOR GATE; MAHER N/A
AS-BUILT REVISION PLANS ARE REQ'D TO INCLUDE THE 2-WAY ELEVATOR LANDING COMMUNICATION SYSTEM UNDER THE FIRE ALARM ADDENDUM 7 PLANS PRIOR TO FINALIZING THIS PERMIT.  (REF #200711309386-S7) COMPLY W/ MAHER ORDINANCE IS NOT REQUIRED.  
ERECT AN ILLUMINATE SINGLE FACED SIGN ON EXISTING CANOPY. N/A MAHER ORDINANCE
COMPLIANCE WITH SOFT STORY SEISMIC RETROFIT-2012 IEBC APPENDIX A-4. MAHER ORDINANCE N/A.
NORTH HALL MDF ROOM 114 - (1) SMOKE DET.  (7) DUCT SMOKE DET  (8) CONTROL RELAYS  (8) DUAL MONITOR MODULES  (2) SPEAKER/STROBES  (1) BATTERY ENCLOSURES W/(2) BATTERIES. N/A MAHER ORDINANCE
BASEMENT 4  3  2  1  LEVEL 1  2  5 THROUGH 30 AND ROOF.  132 EVACUATION SIGNS MAHER - NOT REQ'D
14TH FLOOR T.I. INCLUDE BUILD OUT OF AN EXISTING VACANT SPACE. SELECTIVE DEMOITION OF NON-STRUCTURE NON-RATED ELEMENTS  NEW NON STRUCTURAL PARTITIONS  NEW CEILING AND LIGHTING  NEW FINISHES  NEW DOORS  FRAMS AND HARDWARE. MAHER N/A. MEP/sprinklers separate permit.
INTERIOR SOFT DEMO IN